### Hints for homework with the commands2 library - 2021 1231 CJH
---

### What is already in the template
* The robotpy example (hatch) has a working drivetrain (drivesusbsystem.py) and a drivetrain sim
* It has a robotcontainer.py that can show you how to 
  * import and instantiate subsystems
  * bind commands to joystick buttons
* The template already has several commands written in the comands folder
  * look through them and understand what they do
  * use these as templates to help you make your own
* Common mistakes
  * not getting an intial value or resetting the encoder / gyro before the action if you are tracking
  * not knowing what the encoders are reporting as it moves - use the sim to see what is changing as you move
  * look at other examples for ideas (the ramsete example has the right setup for encoder units)
  

---
### #1: Driving for a set amount of time

General hints:
* make a new command.  the easiest way is to copy one of the simple ones and rename it
* you can use the command drivedistance.py for help
  * that one drives a distance, so it is checking the encoder to see if it should end
  * if you just want to drive for a time, you can set the command timeout and check for timed out in the isFinished() part of the command
  * they took away the isTimedOut and setTimeOut functions in commands2
  * so you should get the time when you initialize and see if the difference is greater than the time you wanted to drive
 

In [1]:
import wpilib
import commands2

In [2]:
drive_time = 2 # for instance, assign to self.drive_time in init

# in initialize stage of command
start_time = wpilib.Timer.getMatchTime()

# in the isFinished stage of the command
def isFinished():
    elapsed_time = wpilib.Timer.getMatchTime() - start_time
    if elapsed_time > drive_time:
        return True
    else:
        return False

---
### #2: Driving for a set distance with a PID

General hints:
* make a new command.  the easiest way is to copy one of the simple ones and rename it
* you can use the command drivedistance.py for help
  * that one drives a distance, so it is checking the encoder to see if it should end
  *  let's not do that.  just let it run until the button is let go so you can see it **overshoot** and recover
  * but instead of just running a fixed speed to `self.robot.drivetrain.arcade_drive(1,0)`, you need to use a PID.  You could write a custom one, like in the 2021 autonomous_drive_pid, or use the wpilib's built in one 
  * you probably want to **clamp**  the output between -1 and 1 - although the max output of the drive will do that for you - it's still good practice

In [29]:
from wpilib.controller import PIDController

In [30]:
# in initialize
setpoint = 2 # or whatever, pass to init and assign to self.setpoint
kp = 0.5  # instead of setting, can get from dashboard
ki = 0.0
kd = 0.0
test_controller = PIDController(Kp=kp, Ki=ki, Kd=kd)
test_controller.setSetpoint(setpoint)

# in execute now you calculate the PID value
measured_distance = 0.5 # self.drive.getAverageEncoderDistance()
output = test_controller.calculate(measured_distance)
#self.drive.arcadeDrive(output, 0)  # drives straight

# in isFinished() you probably just want to return False if you called it with a whileHeld() on the button

---
### #3: Rotating for a set angle with a PID

* it's the same as #2 for the most part
* you now need to use the gyro instead of the wheel encoder
* but there is no gyro in the template_auto project
  * look in the ramsete project for how to add a gyro and a get_heading() function 
  


---
### Well-written commands - components
* \_\_init__() - the constructor, called exactly once per command instance (although you could instantiate the same command multiple times - like binding two different buttons to the same command)
  * this is where you set up class variables that you need in the other member functions
* initialize() - not the same as \_\_init__.  This is called every time the command is run, not just when it is instantiated 
    * this is where you print out start times - friendly debugging messages, etc
    * also where you update a command that may need input from sensors or the dashboard
* execute() - run every time the robot goes through the main loop
  * here is where you do calculations, make measurements, send power to motors, etc
* isFinished() - return True if you want the command to end, False if not
  * often we check a button still pressed, or if a sensor has reached a position
  * for instant commands (fire once and forget) you just `return False`
* end() - what to do when the command ends - either killed by scheduler or isFinished() returns True
  * if you had a start message, you can put your finish message here
  * if motors, etc need to go to a final state, do that here
* interrupted () - was the same as end() in old versions, now end() is passed a parameter called interrupted
  * so in end() check to see if interrupted==True: if so, the command was interrupted (another command took control, or the scheduler killed it for another reason), otherwise it ended by isFinished() returning True.  

#### tips - the DriveDistance from the hatch example
(I had to comment out a few lines for things that can't load from Jupyter)

In [4]:
# command as provided - i had to commend a few things out to get it to run
import commands2
#from subsystems.drivesubsystem import DriveSubsystem


class DriveDistance(commands2.CommandBase):
    def __init__(self, inches: float, speed: float, drive) -> None:
        super().__init__()
        self.distance = inches
        self.speed = speed
        self.drive = drive
        self.addRequirements(drive)

    def initialize(self) -> None:
        self.drive.resetEncoders()
        self.drive.arcadeDrive(self.speed, 0)

    def execute(self) -> None:
        self.drive.arcadeDrive(self.speed, 0)

    def end(self, interrupted: bool) -> None:
        self.drive.arcadeDrive(0, 0)

    def isFinished(self) -> bool:
        return abs(self.drive.getAverageEncoderDistance()) >= self.distance


##### setting match time
* Timer.getMatchTime() is no good, since it is supposed to be a countdown
* Timer.getFPGATimestamp is basically just like time.time(), so either is fine as long as we are consistent
* robot_container should have a .start_time member, and a get and set function for it.\
  * then the robot autonomousInit() and teleopInit() should call the set function in robot container

In [16]:
from wpilib import Timer
Timer.getFPGATimestamp()
import time
time.time()

1641056685.7789464

##### things for robot container - set up an start_time so we can track when we were enabled 

In [19]:
import time

#self.start_time = time.time()  # goes in init

def set_start_time(self):  # call in teleopInit and autonomousInit in the robot
    self.start_time = time.time()
def get_start_time(self):  # call when we want to know the start/elapsed time for status and debug messages 
    return self.start_time

##### things for \_\_init__() only called once per button assignment
* use self.setName() so we have a name we can call later
  * e.g `self.setName('halve_drive_speed')`
  * alternately, could also use `self.__class__.__name__` in those name calls if you'd like to skip the above
* pass a robot container so you can reference the rest of the robot's systems if you need to


In [27]:
class SomeRandomCommand(commands2.CommandBase):
    def __init__(self, container) -> None:
        super().__init__()
        self.setName('some_random_command')  # what is returned by self.getName() - used to be able to pass this to super's init but not in v2
        self.robot_container = container

##### things for initialize() in commands - called every time the command is run

In [ ]:
# how we used to do it - we started a timer when the robot was enabled
# have to figure out how to make this avaialable since we use robotcontainer now
"""Called just before this Command runs the first time."""
import time  # belongs at the top of the file with the rest of the imports

self.start_time = round(time.time() - self.robot_container.get_start_time(), 1)
print("\n" + f"** Started {self.getName()} at {self.start_time} s **", flush=True)
SmartDashboard.putString("alert", f"** Started {self.getName()} at {self.start_time:2.2f} s **")

##### things for end() in commands

In [ ]:
"""Called once after isFinished returns True or command is interrupted"""
end_time = round(time.time() - self.robot_container.get_start_time(), 1)
message = 'Interrupted' if interrupted else 'Ended'
print(f"** {message} {self.getName()} at {end_time} s after {round(end_time-self.start_time,1)} s **")
SmartDashboard.putString("alert", f"** {message} {self.getName()} at {end_time} s after {round(end_time-self.start_time,1)} s **")